# Introduction 

Following Code Uses PyLag, offline Particle Tracking model and FVCOM outputs that are Downloaded from the following for 2023 https://noaa-nos-ofs-pds.s3.amazonaws.com/index.html#lmhofs/netcdf
and Computes Lagrangian Particle Tracking for particles that are release at the mouth of Lake Huron's stream watersheds.

in the local directory they are placed here
FVCOME files are in this location 
#S:\Data\External_Models\Outputs\GLCFS\LakeHuron

## Required imports

In [1]:
# File system and configuration management
import os
import glob
import configparser
import datetime

# Data handling and processing
import numpy as np
import pandas as pd
from collections import namedtuple
from datetime import timedelta, datetime

# NetCDF data handling
from netCDF4 import Dataset
from cftime import num2pydate
import xarray as xr

# Visualization: general plotting, Cartopy, and Matplotlib utilities
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import matplotlib.cm as cm

# FVCOM-specific visualization and utility tools
from pylag.processing.plot import FVCOMPlotter, create_figure, colourmap
from pylag.processing.utils import get_grid_bands
from pylag.grid_metrics import create_fvcom_grid_metrics_file

# Regridding, viewing, and garbage collection utilities
from pylag.regrid import regridder
from pylag.processing.ncview import Viewer
import gc


# Inputs

In [2]:
#FVCOM_DIR='/home/abolmaal/Data/FVCOMEDATA'.format(os.environ['HOME']) 
# fvcom model directory
FVCOM_DIR = '/mnt/hydroglg/Data/External_Models/Outputs/GLCFS/LakeHuron/2023'

# Create run directory
cwd = os.getcwd()
# Create run directory
MODELLING_DIR = '/home/abolmaal/modelling/FVCOM/Huron'.format(cwd)
try:
    os.makedirs(MODELLING_DIR)
except FileExistsError:
    pass


# Create input sub-directory for input files
input_dir = '{}/input'.format(MODELLING_DIR)
try:
    os.makedirs(input_dir)
except FileExistsError:
    pass


# input file to create grid metrics
fvcom_file_name = os.path.join(FVCOM_DIR, 'nos.lmhofs.fields.n000.20230101.t00z.nc')

# The file listing the location of open boundary nodes

obc_file_name = os.path.join(MODELLING_DIR,'input', 'obc.dat')



# initial position of the particles
#initial_position_file = os.path.join(MODELLING_DIR, 'input', 'initial_position', 'initial_positions_releasezone_intersection_multigroup_2_lastrevised.dat')
initial_position_file = os.path.join(MODELLING_DIR, 'input', 'initial_position', 'initial_positions_releasezone_intersection_multigroup_middleplume_final.dat')


# config file
config_file_name = os.path.join(MODELLING_DIR, 'config_files', 'Huron_Senseflux_Seasonal.cfg')


In [13]:
# read tje fvcom_file_name as xarray dataset
fvcom_ds = xr.open_dataset(fvcom_file_name, decode_times=False)
fvcom_ds
# prin the variable u and v
# print(fvcom_ds['u'].values)
# print(fvcom_ds['v'].values)


<xarray.Dataset> Size: 184MB
Dimensions:             (nele: 171377, node: 90806, siglay: 20, siglev: 21,
                         three: 3, time: 1, maxnode: 11, maxelem: 9, four: 4)
Coordinates:
    lon                 (node) float32 363kB ...
    lat                 (node) float32 363kB ...
    lonc                (nele) float32 686kB ...
    latc                (nele) float32 686kB ...
    siglay              (siglay, node) float32 7MB ...
    siglev              (siglev, node) float32 8MB ...
  * time                (time) float32 4B 1.826e+03
Dimensions without coordinates: nele, node, three, maxnode, maxelem, four
Data variables: (12/58)
    nprocs              int32 4B ...
    partition           (nele) int32 686kB ...
    x                   (node) float32 363kB ...
    y                   (node) float32 363kB ...
    xc                  (nele) float32 686kB ...
    yc                  (nele) float32 686kB ...
    ...                  ...
    wet_cells_prev_ext  (time, nele) int32 686kB ...
    aice                (time, node) float32 363kB ...
    vice                (time, node) float32 363kB ...
    tsfc                (time, node) float32 363kB ...
    uuice               (time, nele) float32 686kB ...
    vvice               (time, nele) float32 686kB ...
Attributes: (12/15)
    title:                       LMHOFS
    institution:                 School for Marine Science and Technology
    source:                      FVCOM_4.3
    history:                     model started at: 01/01/2023   02:09
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    Conventions:                 CF-1.0
    ...                          ...
    River_Forcing:               THERE ARE 12 RIVERS IN THIS MODEL.\nRIVER IN...
    GroundWater_Forcing:         GROUND WATER FORCING IS OFF!
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    Surface_Wind_Forcing:        FVCOM variable surface Wind forcing:\nFILE N...
    Surface_PrecipEvap_Forcing:  FVCOM periodic surface precip forcing:\nFILE...
    Ice_Model_Forcing:           FVCOM variable surface ice model forcing:\nF...

# Outputs

In [3]:
# # The name of the output file containing the grid metrics
# # create a sub directory for the grid file
# grid_file_dir = f'{input_dir}/gridfile'
# try:
#     os.makedirs(grid_file_dir)
# except FileExistsError:
#     pass
grid_metrics_file_name = f'{input_dir}/gridfile/grid_metrics_huron_senseflux_Seasonal.nc'

# Create output sub-directory
output_dir = '{}/output'.format(MODELLING_DIR)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)



# Main Functions

## 1-Create Grid metrics

#### this part is only need to run one time

In [5]:
#Generate the file
create_fvcom_grid_metrics_file(fvcom_file_name, obc_file_name = obc_file_name,
                            grid_metrics_file_name=grid_metrics_file_name) 

Creating FVCOM grid metrics file /home/abolmaal/modelling/FVCOM/Huron/input/gridfile/grid_metrics_huron_senseflux_Seasonal.nc
INFO - detected longitudes greater than 180.0 in variable lon. Assuming longitude limits are in the range 0 - 360. Correcting these to be in the range -180 to 180.
INFO - detected longitudes greater than 180.0 in variable lonc. Assuming longitude limits are in the range 0 - 360. Correcting these to be in the range -180 to 180.

Calculating element areas ... done
Grid has 51110 nodes on the open boundary


# 2- Config File

### Updating configure file time and name

In [7]:
import datetime

def update_datetime_in_config(config_path, new_start, new_end):
    # Extract year and month from start and end datetime
    start_date = datetime.datetime.strptime(new_start, '%Y-%m-%d %H:%M:%S')
    end_date = datetime.datetime.strptime(new_end, '%Y-%m-%d %H:%M:%S')
    
    start_year = start_date.year
    start_month = start_date.month
    end_year = end_date.year
    end_month = end_date.month
    
    # Format the output file name (e.g., FVCOM_Huron_2223_DecMar)
    month_range = f"{start_date.strftime('%b')}{end_date.strftime('%b')}"  # Abbreviated months (e.g., 'DecMar')
    output_filename = f"FVCOM_Huron_{start_year % 100}{end_year % 100}_{month_range}"

    with open(config_path, 'r') as file:
        lines = file.readlines()

    with open(config_path, 'w') as file:
        for line in lines:
            if line.strip().startswith("start_datetime"):
                file.write(f"start_datetime = {new_start}\n")
            elif line.strip().startswith("end_datetime"):
                file.write(f"end_datetime = {new_end}\n")
            elif line.strip().startswith("output_file"):
                file.write(f"output_file = %(out_dir)s/{output_filename}\n")  # Update the output_file line
            else:
                file.write(line)

### Adjusting config file time and output name

In [9]:
# #start and end datetime
start_datetime = '2023-01-01 00:00:00'
end_datetime = '2023-02-28 00:00:00'
# Update the config file with new start and end datetime
update_datetime_in_config(config_file_name, start_datetime, end_datetime)
print(f"Updated start_datetime, end_datetime, and output_file in {config_file_name}.")

Updated start_datetime, end_datetime, and output_file in /home/abolmaal/modelling/FVCOM/Huron/config_files/Huron_Senseflux_Seasonal.cfg.


### Creating Run configuration

In [12]:
cf = configparser.ConfigParser()
cf.read(config_file_name)

# Start time
print('Start time: {}'.format(cf.get('SIMULATION', 'start_datetime')))

# End time
print('End time: {}'.format(cf.get('SIMULATION', 'end_datetime')))

# Specify that this is a forward tracking experiment
print('Time direction: {}'.format(cf.get('SIMULATION', 'time_direction')))

# We will do a single run, rather than an ensemble run
print('Number of particle releases: {}'.format(cf.get('SIMULATION', 'number_of_particle_releases')))

# Use depth restoring, and restore particle depths to the ocean surface
print('Use depth restoring: {}'.format(cf.get('SIMULATION', 'depth_restoring')))
print('Restore particles to a depth of: {} m'.format(cf.get('SIMULATION', 'fixed_depth')))

# Specify that we are working with FVCOM in cartesian coordinates0
print('Model name: {}'.format(cf.get('OCEAN_DATA', 'name')))
print('Coordinate system: {}'.format(cf.get('SIMULATION', 'coordinate_system')))

# Set the location of the grid metrics and input files
print('Data directory: {}'.format(cf.get('OCEAN_DATA', 'data_dir')))
print('Path to grid metrics file: {}'.format(cf.get('OCEAN_DATA', 'grid_metrics_file')))
print('File name stem of input files: {}'.format(cf.get('OCEAN_DATA', 'data_file_stem')))
      
# Do an advection only run using a RK$ intergration scheme 
print('Numerical method: {}'.format(cf.get('NUMERICS', 'num_method')))
print('Iterative method: {}'.format(cf.get('NUMERICS', 'iterative_method')))

# print velocity calculater
#print('Velocity calculator: {}'.format(cf.get('CONSTANT_SETTLING_VELOCITY_CALCULATOR', 'initialisation_method')))


# print biological process you used
#print('Biological process: {}'.format(cf.get('BIO_MODEL', 'mortality_calculator')))
# print mortality method 
#print('Mortality method: {}'.format(cf.get('FIXED_TIME_MORALITY_CALCULATOR', 'initialisation_method')))

Start time: 2023-01-01 00:00:00
End time: 2023-02-28 00:00:00
Time direction: forward
Number of particle releases: 1
Use depth restoring: True
Restore particles to a depth of: 0.0 m
Model name: FVCOM
Coordinate system: geographic
Data directory: /mnt/hydroglg/Data/External_Models/Outputs/GLCFS/LakeHuron/2023
Path to grid metrics file: /home/abolmaal/modelling/FVCOM/Huron/input/gridfile/grid_metrics_huron_senseflux_Seasonal.nc
File name stem of input files: nos.lmhofs.fields.n000.
Numerical method: standard
Iterative method: Adv_RK4_3D


# I am not using part 3-5

## 3-Setting Mortality

If you use the following config file huron_senseflux_20230103_Seasonal_mortality.cfg, you don't need to run section 5. it is here for demonstration and showing how mortality works.

In [ ]:
# # # Imports
# import numpy as np
# import matplotlib
# from matplotlib import pyplot as plt
# from configparser import ConfigParser

# import pylag.random as random
# from pylag.data_reader import DataReader
# from pylag.particle_cpp_wrapper import ParticleSmartPtr
# from pylag.mortality import get_mortality_calculator
# from pylag.processing.plot import create_figure

# # Ensure inline plotting
# %matplotlib inline

# # Parameters
# seconds_per_day = 86400.

# # Seed the random number generator
# random.seed(10)

# # Create the config
# #cf.add_section('NUMERICS')
# cf.add_section('BIO_MODEL')
# cf.add_section('FIXED_TIME_MORTALITY_CALCULATOR')
# cf.add_section('PROBABILISTIC_MORTALITY_CALCULATOR')
# # We need a data reader to pass to the mortality calculator. It
# # can be used to draw out environmental variables (e.g. temperature)
# # that affect mortality. In both cases below, it isn't used, so we
# # use the base class.
# data_reader = DataReader()

# # Set time stepping params
# n_particles = 1000
# simulation_duration_in_days = 30.0
# time_step = 100
# time_end = simulation_duration_in_days * seconds_per_day
# times = np.arange(0.0, time_end, time_step)

In [ ]:
#  #Helper function in which the model is run and mortality computed
# def run(config, n_particles=1000):
#     """ Run the model to compute mortality through time """

#     # Create the mortality calculator
#     mortality_calculator = get_mortality_calculator(config)

#     # Create the living particle seed
#     particle_set = []
#     for i in range(n_particles):
#         # Instantiate a new particle
#         particle = ParticleSmartPtr(age=0.0, is_alive=True)

#         # Initialise particle mortality parameters
#         mortality_calculator.set_initial_particle_properties_wrapper(particle)

#         # Append it to the particle set
#         particle_set.append(particle)

#     # Store the number of living particles in a list
#     n_alive_arr = []

#     # Run the model
#     n_alive = n_particles
#     for t in times:
#         n_alive_arr.append(n_alive)

#         n_deaths = 0
#         for particle in particle_set:
#             if particle.is_alive:
#                 mortality_calculator.apply_wrapper(data_reader, t, particle)
#                 if particle.is_alive == False:
#                     n_deaths += 1
#             particle.set_age(t)

#         n_alive -= n_deaths

#     return n_alive_arr

## 4-FixedTimeMortalityCalculater

In [ ]:
# # Specify a fixed time mortality calculator
# cf.set('BIO_MODEL', 'mortality_calculator', 'fixed_time')

# # 1) Fixed time scenario
# # Sharp_2021 suggerst 10 days fpr N uptake in coastal wetlands
# age_of_death_in_days = 10.
# cf.set('FIXED_TIME_MORTALITY_CALCULATOR', 'initialisation_method', 'common_value')
# cf.set('FIXED_TIME_MORTALITY_CALCULATOR', 'common_value', str(age_of_death_in_days))
# n_alive_common_value = run(cf)

In [ ]:
# # 2) Uniform Random 
# minimum_bound = 8.
# maximum_bound = 12.
# cf.set('FIXED_TIME_MORTALITY_CALCULATOR', 'initialisation_method', 'uniform_random')
# cf.set('FIXED_TIME_MORTALITY_CALCULATOR', 'minimum_bound', str(minimum_bound))
# cf.set('FIXED_TIME_MORTALITY_CALCULATOR', 'maximum_bound', str(maximum_bound))
# n_alive_uniform_random = run(cf)

In [ ]:
# # 2) Gaussian random
# mean = 10.
# standard_deviation = 1.
# cf.set('FIXED_TIME_MORTALITY_CALCULATOR', 'initialisation_method', 'gaussian_random')
# cf.set('FIXED_TIME_MORTALITY_CALCULATOR', 'mean', str(mean))
# cf.set('FIXED_TIME_MORTALITY_CALCULATOR', 'standard_deviation', str(standard_deviation))
# n_alive_gaussian_random = run(cf)
# # Set the bio time step
# cf.set('NUMERICS', 'time_step_bio', str(time_step))

In [ ]:
# Plot
# font_size = 10
# fig, ax = create_figure(figure_size=(20, 20), font_size=font_size)
# plt.plot(times/seconds_per_day, n_alive_common_value, 'b', label='common_value')
# plt.plot(times/seconds_per_day, n_alive_uniform_random, 'r', label='uniform_random')
# plt.plot(times/seconds_per_day, n_alive_gaussian_random, 'g', label='gaussian_random')
# # Set the bio time step
# plt.ylabel('Living individuals (-)', fontsize=font_size)
# plt.xlabel('Time (d)', fontsize=font_size)

# # Add legend
# plt.legend()

## 5-ProabilisticMortalityCalculator

The mortality calculator kills particles at a rate 
, where 
 is a fixed mortality rate which is set in the run configuraiton file and 
 is the model time step for biological processes. The model computes a uniform random deviate in the range (0, 1). If the number is less than the computed death rate, the particle is killed. Below, we create a population of 
 individuals. We apply a death rate of 
 per day and use a time step of 
 seconds. The model is run forward for 
 days and the number of living individuals plotted as a function of time. The result is compared with a simple analytical solution of exponential decay

In [ ]:

# # Specify a probabilistic mortality calculator
# cf.set('BIO_MODEL', 'mortality_calculator', 'probabilistic')

# # Set the death rate - currently the same for all particles.
# death_rate_per_day = 0.1
# cf.set('PROBABILISTIC_MORTALITY_CALCULATOR', 'death_rate_per_day', str(death_rate_per_day))

# # Set the bio time step
# cf.set('NUMERICS', 'bio_time_step', str(time_step))

# # Number of particles
# n_particles = 1000

# # Run the model
# n_alive_numeric = run(cf, n_particles=n_particles)

# # Compute the equivalent analytical solution
# death_rate_per_second = death_rate_per_day / seconds_per_day
# n_alive_analytic = n_particles * np.exp(-death_rate_per_second * times)

# # Plot
# font_size = 10
# fig, ax = create_figure(figure_size=(20, 20), font_size=font_size)
# plt.plot(times/seconds_per_day, n_alive_numeric, 'b', label='numeric')
# plt.ylabel('Living individuals (-)', fontsize=font_size)
# plt.xlabel('Time (d)', fontsize=font_size)

# # Add equivalent analytical solution
# plt.plot(times/seconds_per_day, n_alive_analytic, 'r', label='analytic')

# # Add legend
# plt.legend()

# 3-Run the model 

In [13]:
# chapck the path doesnt have a pylag.cfg file 
# Check if pylag.cfg exists in the output directory and delete it if present
pylag_cfg_path = os.path.join(output_dir, 'pylag.cfg')

# If the file exists, delete it
if os.path.exists(pylag_cfg_path):
    print(f"Deleting existing pylag.cfg in {output_dir}")
    os.remove(pylag_cfg_path)

In [14]:
# Set up configuration options
cf.set('OCEAN_DATA', 'data_dir', FVCOM_DIR)
cf.set('OCEAN_DATA', 'grid_metrics_file', grid_metrics_file_name)

# Directory where the simulation outputs will be saved
out_dir = f"{MODELLING_DIR}/output"
cf.set('GENERAL', 'out_dir', out_dir)

# Save a copy in the simulation directory
with open(f"{MODELLING_DIR}/pylag.cfg", 'w') as config:
    cf.write(config)
    
# Save a copy in the output directory
print(f"Updated configuration and saved to {pylag_cfg_path}")


Updated configuration and saved to /home/abolmaal/modelling/FVCOM/Huron/output/pylag.cfg


In [15]:
# Change to the run directory
os.chdir(f"{MODELLING_DIR}")

# Run the model
!{"python -m pylag.main -c pylag.cfg"}

# Return to the cwd
os.chdir(cwd)


Starting ensemble member 1 ...
Progress:
100% |###########################################|
